### Quantum Fourier Transform.
#### In this colab:
* We will take a quick overview of QFT. \
* implement QFT manually using QCSimulator lib. \
* Take a look on build-in `qft()` and `qft_rev()` functions inside QCSimulator lib.

### 1. Quick overview:
Let's take a look at how the QFT implementation looks like in general.
![QFT](https://drive.google.com/uc?id=1RtLz3PxBQVkboksMAn2k23O-960khOY4) 
Where $\phi_n = e^{\dfrac{2i\pi}{2^n}}$ \
For better understanding we also can treat H gate as a $e^{q_0 i\pi}$ apllied on $|1⟩$ state of the qubit.
\
H gate is necessary before any of the CROT gates, to ensure that a qubit is in a superposition state. Then, the CROT gate applies an additional phase to the $|1⟩$ component of a qubit. \
What happens in QFT is we kind of swapping from position space to momentum space so we can analyze and see our wavefunction from a different "angle" and apply some other gates to that state, then reverse it back using inverse QFT. \
\
Basically, QFT is a DFT applied to the quantum bits. \
The DFT as well as QFT can help us find a period of a function. For factorization problem we need to find the period for $f(x) = a^x \mod M$ where M is a number to factorize. \
QFT, like all the operations in QC, is reversible. When inverting the QFT we have to notice that the angles become negative (to kind of rotate to initial position).

### 2. Implementation. 
Let's now create a circuit and manually implement QFT using CROT and H gates from our lib.

In [ ]:
#instaling the library
!git clone https://github.com/katolikyan/QCSimulator.git
!pip3 install QCSimulator/

In [2]:
import qcsimulator as qcs
import numpy as np

A circuit we are going to implement will look like this, we took a little bit bigger circuit to feel the angle relations between CROT gates.
We will apply an X gate to the second qubit just to trigger controlled phase rotation gates and take a look at output state vector.
At the 2nd block of code, we will apply inverse QFT.

In [3]:
# Our sicruit is going to look like this:
# |0> ----H--R--R--R--------------------
#            |  |  |
# |0> -------*--|--|--H--R--R-----------
#               |  |     |  |
# |0> --X-------*--|-----*--|--H--R-----
#                  |        |     |
# |0> -------------*--------*-----*--H--

circuit = qcs.circuit_init(4) # circuit initialization with 4 qubits.
circuit.x(2) # X on the third qubit
result = circuit.execute()
sv_initial = result.get_state_vector()
print("----- Initial state vector -----")
print(sv_initial, "\n")

#QFT
# qubit 0
circuit.h(0)
circuit.crot(1, 0, np.pi / 2)
circuit.crot(2, 0, np.pi / 4)
circuit.crot(3, 0, np.pi / 8)
# qubit 1
circuit.h(1)
circuit.crot(2, 1, np.pi / 4)
circuit.crot(3, 1, np.pi / 8)
# qubit 2
circuit.h(2)
circuit.crot(3, 2, np.pi / 8)
# qubit 3
circuit.h(3)

result_qft = circuit.execute()
sv_qft = result_qft.get_state_vector()
print("----- State vector after QFT -----")
print(sv_qft)

----- Initial state vector -----
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j] 

----- State vector after QFT -----
[ 2.50000000e-01+0.j         1.76776695e-01+0.1767767j
  1.76776695e-01+0.1767767j  5.55111512e-17+0.25j
 -2.50000000e-01+0.j        -1.76776695e-01-0.1767767j
 -1.76776695e-01-0.1767767j -5.55111512e-17-0.25j
  2.50000000e-01+0.j         1.76776695e-01+0.1767767j
  1.76776695e-01+0.1767767j  5.55111512e-17+0.25j
 -2.50000000e-01+0.j        -1.76776695e-01-0.1767767j
 -1.76776695e-01-0.1767767j -5.55111512e-17-0.25j     ]


In [4]:
#inverse QFT manually. Notice that now we will have negative angles.
# ---H--R--R--R--------------------  --------------------R--R--R--H---
#       |  |  |                                          |  |  |
# ------*--|--|--H--R--R-----------  -----------R--R--H--|--|--*------
#          |  |     |  |                        |  |     |  |  
# ---------*--|-----*--|--H--R-----  -----R--H--|--*-----|--*---------
#             |        |     |            |     |        | 
# ------------*--------*-----*--H--  --H--*-----*--------*------------

# qubit 3
circuit.h(3)
# qubit 2
circuit.crot(3, 2, -np.pi / 8)
circuit.h(2)
# qubit 1
circuit.crot(3, 1, -np.pi / 8)
circuit.crot(2, 1, -np.pi / 4)
circuit.h(1)
# qubit 0
circuit.crot(3, 0, -np.pi / 8)
circuit.crot(2, 0, -np.pi / 4)
circuit.crot(1, 0, -np.pi / 2)
circuit.h(0)

result_qft_rev = circuit.execute()
sv_qft_rev = result_qft_rev.get_state_vector()
print("----- State vector after QFT reversed -----")
print(sv_qft_rev)
# lets cut the garbage caused by floats calculations.
print("\n----- State vector after QFT reversed (rounded decimals=16) -----")
print(np.around(sv_qft_rev, decimals=16))

# To validate not just visually let's compare it using np.testing
assert(np.testing.assert_allclose(sv_initial, sv_qft_rev, \
                                  rtol=1e-8, atol=1e-8)) == None

----- State vector after QFT reversed -----
[ 1.68365798e-18+8.12941988e-18j -1.68365798e-18-8.12941988e-18j
  1.68365798e-18+1.68365798e-18j  1.79424977e-17-1.68365798e-18j
  1.00000000e+00+0.00000000e+00j -8.90187929e-18+0.00000000e+00j
 -2.79639290e-18+1.09258128e-17j -3.64559186e-17-1.09258128e-17j
 -1.49797999e-18+1.78248605e-18j  3.48696986e-19+6.93601429e-20j
  8.39982398e-19+3.92457343e-19j  6.71794186e-19-4.21925269e-19j
  1.16616167e-17-1.01138088e-18j  9.93005002e-19+1.48613701e-18j
  8.88250476e-19-3.88802594e-18j -4.38115794e-18+1.59089153e-18j]

----- State vector after QFT reversed (rounded decimals=16) -----
[ 0.+0.j -0.-0.j  0.+0.j  0.-0.j  1.+0.j -0.+0.j -0.+0.j -0.-0.j -0.+0.j
  0.+0.j  0.+0.j  0.-0.j  0.-0.j  0.+0.j  0.-0.j -0.+0.j]


We can see that the output after transformation is exactly the same except
that we have some confusing differenses on signs, but it appears only on zeroes and caused because of not perfect precision calculations. So we should not really care about it. Just a little experiment to prove that it doen't affect anything.

In [5]:
a = -0.-0.j
b = -0.+0.j
c = +0.-0.j
d = +0.+0.j
e = 0.+0.j
f = 0.-0.j
print(a == b == c == d == e == f) # expecting True
print(a - 1.-1j == b - 1.-1j == c - 1.-1j ==\
      d - 1.-1j == e - 1.-1j == f- 1.-1j) # expecting True

a = 1.-0.j
b = 1.+0.j
print(a == b) # expecting True

a = 1.-0.j
b = -1.-0.j
print(a == b) # expecting False

True
True
True
False


### 3. `qft` and `qft_rev`.
Let's take a look at the function's code inside the library.
We can see that they accept 2 indexes which represent between which qubits we want to apply QFT. notice that both indexes are included.

In [6]:
  # The method from the class Circuit to apply QFT to the given range of qubits.
  def qft(self, first: int, last: int) -> None:
    self._check_input(first, last, only_positive=True)  # currently only positive indcies are acceptable.
    for crnt in range(first, last + 1):                 # iterating from first qubit to the last one.
      self.h(crnt)                                        # applying a Hadamard on each step.
      for i in range(crnt, last):                         # iterating from crnt qubit to the last one.
        angle = np.pi / (2 ** (i + 1))                      # calculating the angle for CROT. 
        self.crot(i + 1, crnt, angle)                       # applying CROT to the crnt qubit. 

  # The method from the class Circuit to apply reversed QFT to the given range of qubits.
  # The logic is exactly the same except we start from the last qubit going up.
  def qft_rev(self, first: int, last: int) -> None: 
    self._check_input(first, last, only_positive=True)  # currently only positive indcies are acceptable.
    for crnt in reversed(range(first, last + 1)):       # iterating from first qubit to the last one.
      for i in reversed(range(crnt, last)):               # iterating from crnt qubit to the last one.
        angle = -1 * np.pi / (2 ** (i + 1))                 # calculating the opposite angle for CROT.
        self.crot(i + 1, crnt, angle)                       # applying CROT to the crnt qubit.
      self.h(crnt)                                        # applying a Hadamard on each step.

Let's validate that our `qft_rev` indeed returns us the initial state.

In [7]:
# creating a circuit and getting state vector before QFT.
circuit = qcs.circuit_init(3)
circuit.x(0)
circuit.h(0)
circuit.x(1)
circuit.h(1)
circuit.x(2)
circuit.h(2)
sv_initial = circuit.execute().get_state_vector()

# applying QFT and getting state vector
circuit.qft(0, 2)
sv_qft = circuit.execute().get_state_vector() 

# applying reverse QFT and getting state vector.
# Expecting to get the same vector before applying QFT.
circuit.qft_rev(0, 2)
sv_qft_rev = circuit.execute().get_state_vector()

print("----- Initial SV -----")
print(sv_initial, "\n")

print("----- SV after QFT -----")
print(sv_qft, "\n")

print("----- SV after reverse QFT -----")
print(np.around(sv_qft_rev, decimals=5))

# To validate not just visually let's compare vectors using np.testing
assert(np.testing.assert_allclose(sv_initial, sv_qft_rev, \
                                  rtol=1e-8, atol=1e-8)) == None

----- Initial SV -----
[ 0.35355339+0.j -0.35355339+0.j -0.35355339+0.j  0.35355339+0.j
 -0.35355339+0.j  0.35355339+0.j  0.35355339+0.j -0.35355339+0.j] 

----- SV after QFT -----
[ 1.90505075e-52+0.00000000e+00j -1.03553391e-01-2.50000000e-01j
  9.11379035e-20-2.20026363e-19j  5.00000000e-01-2.74444113e-17j
  1.92058443e-35+0.00000000e+00j  6.03553391e-01-2.50000000e-01j
  5.31190629e-19+2.20026363e-19j  9.03125231e-17+5.00000000e-01j] 

----- SV after reverse QFT -----
[ 0.35355+0.j -0.35355-0.j -0.35355-0.j  0.35355+0.j -0.35355+0.j
  0.35355+0.j  0.35355+0.j -0.35355-0.j]


---
### Experiments with QFT:
In the following colab I am trying to make some periodic functions, play with QFT and implement quantum phase estimation algorithm.\
Code here is in progress and may not work properly and may not be clear, don't take it too serios :) \
https://colab.research.google.com/drive/1IaP_d-LEr-BPWCaQhCpZRkjGf9uABgKx

--- 
#### Thank you!
All info and updates for the lib can be found on the project's repo on GitHub: \
https://github.com/katolikyan/QCSimulator \
Do not hesitate to open an issue and contribute :)
